Install Berthopic

In [3]:
from bertopic import BERTopic
import pandas as pd
import nltk

In [4]:
import os
import docx

In [ ]:
f = open("rogan.txt", "r")
text = f.read()
text

In [7]:
sentences = text.split('.')

In [ ]:
sentences

## First cleaning

In [9]:
from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize 

def clean (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return lemmatized

In [10]:
sentences_clean = []
counter = 0
for item in sentences:
    sentences_clean.append(clean(sentences[counter]))
    counter = counter + 1

counter = 0
for item in sentences_clean:
    sentences_clean[counter] = ' '.join(sentences_clean[counter])
    counter = counter + 1
print(counter)

1359


In [26]:
from transformers.pipelines import pipeline

hf_model = pipeline("feature-extraction", model="distilbert-base-cased")

topic_model = BERTopic()

topic_model = BERTopic(language="english", embedding_model=hf_model,calculate_probabilities=True, verbose=True, n_gram_range=(1, 3), nr_topics="auto")

topics, probs = topic_model.fit_transform(sentences_clean)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 1359/1359 [00:28<00:00, 47.53it/s]
2022-09-07 14:20:37,220 - BERTopic - Transformed documents to Embeddings
2022-09-07 14:20:40,251 - BERTopic - Reduced dimensionality
2022-09-07 14:20:40,292 - BERTopic - Clustered reduced embeddings
2022-09-07 14:20:41,

NameError: name 'probs' is not defined

In [61]:
topic_model.get_topic_info()['Name'][0:11]
#topics['Name']
#df['Name']

0                          -1_people_america_book_think
1                 0_ayahuasca_cannabis_dmt_psychedelics
2     1_archaeologist_natural history museum_san_arc...
3                 2_authority_internet_fear_nationalism
4     3_rainforest_evidence_bean farm_million square...
5                4_beautiful_need_done_looking hard wow
6     5_suddenly oh_oh applause_suddenly oh applause...
7                          6_idea_debate_matter_one day
8     7_civilization_old world_advanced civilization...
9     8_serpent mound_summer solstice_head_natural r...
10    9_neanderthal_denisovans_anatomically modern h...
Name: Name, dtype: object

In [23]:
if len(topics) > 2:
    print("true")

In [28]:
words_to_be_deleted = []
for i in range(2,len(topics)-1):
        words_to_be_deleted.append(topic_model.get_topic(i)[0][0].split(' ')[0])
words_to_be_deleted

[]

## second cleaning

In [29]:
from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize 

def clean2(text):
    tokenized = word_tokenize(text) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = words_to_be_deleted # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    return without_stopwords

In [30]:
sentences_clean2 = []
counter = 0
for item in sentences_clean:
    sentences_clean2.append(clean2(item))
    counter = counter + 1

counter = 0
for item in sentences_clean2:
    sentences_clean2[counter] = ' '.join(item)
    counter = counter + 1
print(counter)

1359


## Final Bertopic model

In [11]:
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
topic_model = BERTopic(embedding_model=sentence_model, language='english', diversity = 0.4, top_n_words=10, n_gram_range=(1, 3), min_topic_size=6)
topics, probabilities = topic_model.fit_transform(sentences_clean2)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [19]:
len(probabilities)

1359

In [44]:
topics = topic_model.get_topic_info()
topics.iloc[1:11]

,Topic,Count,Name
1,0,71,0_ayahuasca_cannabis_dmt_psychedelics
2,1,51,1_archaeologist_natural history museum_san_arc...
3,2,44,2_authority_internet_fear_nationalism
4,3,35,3_rainforest_evidence_bean farm_million square...
5,4,30,4_beautiful_need_done_looking hard wow
6,5,29,5_suddenly oh_oh applause_suddenly oh applause...
7,6,28,6_idea_debate_matter_one day
8,7,28,7_civilization_old world_advanced civilization...
9,8,26,8_serpent mound_summer solstice_head_natural r...
10,9,24,9_neanderthal_denisovans_anatomically modern h...


In [50]:
topics.shape[0]

58

In [49]:
len(topics)

58

In [48]:
if len(topics) > 3:
    print("a")

a


In [46]:
words_to_be_deleted = []
if len(topics) > 3:
    for k in range(2,len(topics)-1):
        words_to_be_deleted.append(topic_model.get_topic(k)[0][0])
words_to_be_deleted

['authority',
 'rainforest',
 'beautiful',
 'suddenly oh',
 'idea',
 'civilization',
 'serpent mound',
 'neanderthal',
 'dna',
 'blue stone',
 'know called',
 'orion',
 'ice age',
 'hundred year',
 'torrid meteor stream',
 'mayan',
 'bimini road',
 'map',
 'burn',
 'meme',
 'solid information',
 'talk event',
 'back image',
 'clovis first',
 'consciousness',
 'specie',
 'civilization',
 'well say mean',
 'barnes noble',
 'scale',
 'rainforest',
 'yeah really still',
 'music',
 'america',
 'tepe',
 'published nature',
 'circle',
 'gatherer',
 'air burst',
 'cosmic impact',
 'feathered serpent',
 'sort advanced',
 'stephen',
 'population',
 'conclusion',
 'journey',
 'technological feat longitude',
 'humongous crater',
 'running middle',
 'interview earn even',
 'tech look',
 'physical',
 'time lot like',
 'go find',
 'nothing reason']

In [39]:
topic_model.get_topic(1)[1][0]

'natural history museum'

In [43]:
counter = 0
list = []
for k in range(10):
    list_temp = []
    for i in range(1):
        list_temp.append(topic_model.get_topic(k)[i][0])
    list.append(list_temp)
list

[['ayahuasca'],
 ['archaeologist'],
 ['authority'],
 ['rainforest'],
 ['beautiful'],
 ['suddenly oh'],
 ['idea'],
 ['civilization'],
 ['serpent mound'],
 ['neanderthal']]